<a href="https://colab.research.google.com/github/RusAl84/text_analitic/blob/master/%D0%9C%D0%B0%D1%81%D1%82%D0%B5%D1%80_%D0%BA%D0%BB%D0%B0%D1%81%D1%81_%D0%B2%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D0%B8%D0%B5_%D0%B2_%D1%82%D0%B5%D0%BA%D1%81%D1%82_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA%D1%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

tfidf=tf∗idf

idf=log((N+1)/(Nw+1))+1

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: TfidfVectorizer, рассмотрим его.

In [1]:
texts = "\u041A\u0430\u043A \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0438\u0437\u043C\u0435\u043D\u0438\u0442 \u043D\u0430\u0448\u0443 \u0436\u0438\u0437\u043D\u044C \u0447\u0435\u0440\u0435\u0437 5 \u043B\u0435\u0442 \u041E\u043F\u0443\u0431\u043B\u0438\u043A\u043E\u0432\u0430\u043D\u043E: 23/11/2019\u0410\u0432\u0442\u043E\u0440: \u0410\u043D\u0434\u0440\u0435\u0439 \u041B\u0430\u043F\u0443\u0448\u043A\u0438\u043D\u041A\u0430\u0442\u0435\u0433\u043E\u0440\u0438\u044F: \u041E\u0431\u0437\u043E\u0440\u044B\u041E\u0441\u0442\u0430\u0432\u0438\u0442\u044C \u043A\u043E\u043C\u043C\u0435\u043D\u0442\u0430\u0440\u0438\u0439 Internet of thing \u2013 IOT \u2013 \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u043E\u0431\u043B\u043E\u0436\u043A\u0430 \u0421\u043E\u0434\u0435\u0440\u0436\u0430\u043D\u0438\u0435\t \u0427\u0442\u043E \u0442\u0430\u043A\u043E\u0435 \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u041A\u0430\u043A \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0443\u043F\u0440\u043E\u0449\u0430\u0435\u0442 \u0436\u0438\u0437\u043D\u044C \u041A\u0430\u043A \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F IoT (internet of things) \u0413\u043B\u0430\u0432\u043D\u0430\u044F \u043E\u0441\u043E\u0431\u0435\u043D\u043D\u043E\u0441\u0442\u044C \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0430 \u0432\u0435\u0449\u0435\u0439\xBB  \u041F\u0440\u0438\u043C\u0435\u043D\u0435\u043D\u0438\u0435 IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0432 \u0440\u0430\u0437\u043D\u044B\u0445 \u0441\u0435\u043A\u0442\u043E\u0440\u0430\u0445 \u041E\u0431\u0449\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0439 \u0441\u0435\u043A\u0442\u043E\u0440 \u0438\u043B\u0438 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u043E \u0433\u043E\u0440\u043E\u0434\u0430 \u0410\u0433\u0440\u0430\u0440\u043D\u044B\u0439 \u0441\u0435\u043A\u0442\u043E\u0440 \u0421\u0435\u043A\u0442\u043E\u0440 \u043B\u043E\u0433\u0438\u0441\u0442\u0438\u043A\u0438 \u0421\u0442\u0440\u043E\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u0439 \u0441\u0435\u043A\u0442\u043E\u0440 \u0427\u0442\u043E \u0442\u0430\u043A\u043E\u0435 \xAB\u0443\u043C\u043D\u044B\u0439 \u0434\u043E\u043C\xBB \u0420\u0430\u0437\u0432\u0438\u0442\u0438\u0435 \xAB\u0443\u043C\u043D\u043E\u0439 \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u044B\xBB \u0432 \u0442\u0435\u0447\u0435\u043D\u0438\u0435 \u043F\u044F\u0442\u0438 \u043B\u0435\u0442 \xAB\u0423\u043C\u043D\u044B\u0439 \u0440\u0438\u0442\u0435\u0439\u043B\xBB \u0438\u043B\u0438 \u043A\u0430\u043A \u0431\u0443\u0434\u0443\u0442 \u0432\u044B\u0433\u043B\u044F\u0434\u0435\u0442\u044C \u043C\u0430\u0433\u0430\u0437\u0438\u043D\u044B \u0431\u0443\u0434\u0443\u0449\u0435\u0433\u043E \u041D\u043E\u0441\u0438\u043C\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0423\u043C\u043D\u044B\u0435 \u043C\u0430\u0448\u0438\u043D\u044B \xABInternet of things\xBB \u2014 \u043E\u0436\u0438\u0434\u0430\u043D\u0438\u044F \u0438 \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0441\u0442\u044C \u0427\u0442\u043E \u0442\u0430\u043A\u043E\u0435 \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u044D\u0442\u043E \u2014 \u0441\u0435\u0442\u044C \u0444\u0438\u0437\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u043F\u0440\u0435\u0434\u043C\u0435\u0442\u043E\u0432 (\xAB\u0432\u0435\u0449\u0435\u0439\xBB), \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043E\u0441\u043D\u0430\u0449\u0435\u043D\u044B \u0432\u0441\u0442\u0440\u043E\u0435\u043D\u043D\u043E\u0439 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0435\u0439 \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044F \u0434\u0440\u0443\u0433 \u0441 \u0434\u0440\u0443\u0433\u043E\u043C \u0438 \u0432\u043D\u0435\u0448\u043D\u0435\u0439 \u0441\u0440\u0435\u0434\u043E\u0439. \u041E\u0440\u0433\u0430\u043D\u0438\u0437\u0430\u0446\u0438\u044F \u0442\u0430\u043A\u0438\u0445 \u0441\u0435\u0442\u0435\u0439 \u0440\u0430\u0441\u0441\u043C\u0430\u0442\u0440\u0438\u0432\u0430\u0435\u0442\u0441\u044F \u043A\u0430\u043A \u044F\u0432\u043B\u0435\u043D\u0438\u0435, \u0441\u043F\u043E\u0441\u043E\u0431\u043D\u043E\u0435 \u043F\u0435\u0440\u0435\u0441\u0442\u0440\u043E\u0438\u0442\u044C \u043A\u0430\u043A \u044D\u043A\u043E\u043D\u043E\u043C\u0438\u0447\u0435\u0441\u043A\u0438\u0435, \u0442\u0430\u043A \u0438 \u043E\u0431\u0449\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0435 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u044B \u043F\u043E\u0441\u0440\u0435\u0434\u0441\u0442\u0432\u043E\u043C \u0438\u0441\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u044F \u0438\u0437 \u0447\u0430\u0441\u0442\u0438 \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0439 \u0438 \u043E\u043F\u0435\u0440\u0430\u0446\u0438\u0439 \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E\u0441\u0442\u044C \u0443\u0447\u0430\u0441\u0442\u0438\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430.  \u041A\u0430\u043A \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0443\u043F\u0440\u043E\u0449\u0430\u0435\u0442 \u0436\u0438\u0437\u043D\u044C \u0414\u0430\u043D\u043D\u0430\u044F \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0438\u0440\u0443\u0435\u0442 \u0440\u0430\u0437\u043B\u0438\u0447\u043D\u044B\u0435 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u044B, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043E\u043A\u0440\u0443\u0436\u0430\u044E\u0442 \u043D\u0430\u0441. \u0422\u0430\u043A, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0444\u0443\u043D\u043A\u0446\u0438\u044F \u0432\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u044F \u0438\u043B\u0438 \u0432\u044B\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u044F \u0434\u0432\u0438\u0433\u0430\u0442\u0435\u043B\u044F \u0438 \u043A\u043E\u043D\u0434\u0438\u0446\u0438\u043E\u043D\u0435\u0440\u0430, \u043D\u0430\u0441\u0442\u0440\u0430\u0438\u0432\u0430\u0435\u043C\u044B\u0435 \u043A\u043B\u0438\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0441\u0438\u0441\u0442\u0435\u043C\u044B \u0438\u043B\u0438 \u0434\u043E\u043C\u0430\u0448\u043D\u0438\u0435 \u043A\u043E\u0444\u0435\u043C\u0430\u0448\u0438\u043D\u044B \u0447\u0435\u0440\u0435\u0437 \u0441\u043C\u0430\u0440\u0442\u0444\u043E\u043D. \u0423\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0441\u0432\u044F\u0437\u044B\u0432\u0430\u044E\u0442\u0441\u044F \u043C\u0435\u0436\u0434\u0443 \u0441\u043E\u0431\u043E\u0439 \u0432 \u0435\u0434\u0438\u043D\u0443\u044E \u0441\u0435\u0442\u044C, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u043F\u0440\u0438\u0437\u0432\u0430\u043D\u0430 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0438\u0442\u044C \u043C\u0430\u043A\u0441\u0438\u043C\u0430\u043B\u044C\u043D\u044B\u0439 \u043A\u043E\u043C\u0444\u043E\u0440\u0442 \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0443 \u0438 \u044D\u043A\u043E\u043D\u043E\u043C\u0438\u044E \u044D\u043D\u0435\u0440\u0433\u043E\u0440\u0435\u0441\u0443\u0440\u0441\u043E\u0432, \u0432\u0440\u0435\u043C\u0435\u043D\u0438.  \u041A\u0430\u043A \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F IoT (internet of things) \u0418\u0434\u0435\u044F \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB ( Internet of Things ) \u0437\u0430\u043A\u043B\u044E\u0447\u0430\u0435\u0442\u0441\u044F \u0432\u043E \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0438 \u0432\u0435\u0449\u0435\u0439 \u0441 \u0441\u0435\u0440\u0432\u0435\u0440\u043E\u043C \u0438 \u043C\u0435\u0436\u0434\u0443 \u0441\u043E\u0431\u043E\u0439 \u0431\u0435\u0437 \u0437\u043D\u0430\u0447\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0433\u043E \u0443\u0447\u0430\u0441\u0442\u0438\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430. \u0413\u043B\u0430\u0432\u043D\u044B\u043C \u043E\u0431\u0440\u0430\u0437\u043E\u043C IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u2014 \u044D\u0442\u043E \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u0435 \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430 \u0441 \u0432\u0435\u0449\u0430\u043C\u0438.  \u041F\u0440\u0438\u043D\u0446\u0438\u043F \u0440\u0430\u0431\u043E\u0442\u044B IoT \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u043C\u043E\u0436\u043D\u043E \u043F\u043E\u043D\u044F\u0442\u044C \u0431\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u0435\u0451 \u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u0435:  IoT-\u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430. \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0438\xBB \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u044E\u0442 \u0441\u043E\u0431\u043E\u0439 \u0441\u043E\u0435\u0434\u0438\u043D\u0435\u043D\u043D\u044B\u0435 \u0447\u0435\u0440\u0435\u0437 \u0441\u0435\u0442\u044C \u043F\u0440\u0435\u0434\u043C\u0435\u0442\u044B \u044D\u043B\u0435\u043A\u0442\u0440\u043E\u043D\u0438\u043A\u0438 \u0438 \u0431\u044B\u0442\u043E\u0432\u043E\u0439 \u0442\u0435\u0445\u043D\u0438\u043A\u0438, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0443\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0442\u0441\u044F \u0435\u0434\u0438\u043D\u044B\u043C \u0446\u0435\u043D\u0442\u0440\u043E\u043C. \u0428\u043B\u044E\u0437\u044B. \u041E\u0431\u043B\u0430\u0441\u0442\u044C, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u043F\u043E\u043B\u0443\u0447\u0430\u0435\u0442 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E \u043E\u0442 \u0432\u0437\u0430\u0438\u043C\u043E\u0437\u0430\u0432\u0438\u0441\u0438\u043C\u044B\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432. \u0428\u043B\u044E\u0437\u044B \u043F\u0435\u0440\u0435\u0434\u0430\u044E\u0442 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E \u043E \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044F\u0445, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E \u0432\u044B\u043F\u043E\u043B\u043D\u0438\u0442\u044C \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0443. \u0427\u0430\u0449\u0435 \u0432\u0441\u0435\u0433\u043E \u043E\u043D\u0438 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u0435\u043D\u044B \u043C\u0430\u0440\u0448\u0440\u0443\u0442\u0438\u0437\u0430\u0442\u043E\u0440\u0430\u043C\u0438 \u0438\u043B\u0438 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u043D\u044B\u043C \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u0435\u043C. \u0421\u0435\u0440\u0432\u0435\u0440. \u042D\u0442\u0430 \u043E\u0431\u043B\u0430\u0441\u0442\u044C \u0445\u0440\u0430\u043D\u0435\u043D\u0438\u044F, \u043E\u0431\u0440\u0430\u0431\u043E\u0442\u043A\u0438 \u0438 \u0430\u043D\u0430\u043B\u0438\u0437\u0430 \u043F\u043E\u043A\u0430\u0437\u0430\u043D\u0438\u044F \u0434\u0430\u0442\u0447\u0438\u043A\u043E\u0432. \u0414\u0430\u043D\u043D\u044B\u0435 \u043D\u0430\u043A\u0430\u043F\u043B\u0438\u0432\u0430\u044E\u0442\u0441\u044F \u0438 \u043E\u0431\u0440\u0430\u0431\u0430\u0442\u044B\u0432\u0430\u044E\u0442\u0441\u044F \u0432 \u043E\u0431\u043B\u0430\u043A\u0435 \u0438\u043B\u0438 \u0432 \u0431\u0430\u0437\u0435 \u0432\u0438\u0440\u0442\u0443\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u0441\u0435\u0440\u0432\u0435\u0440\u0430. \u041A\u043B\u0438\u0435\u043D\u0442\u0441\u043A\u0430\u044F \u0447\u0430\u0441\u0442\u044C. \u041E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u043D\u0438\u0435 \u0438 \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u0435 \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0439 \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u043E\u043C \u0447\u0435\u0440\u0435\u0437 \u043C\u043E\u0431\u0438\u043B\u044C\u043D\u043E\u0435 \u0438\u043B\u0438 \u0432\u0435\u0431-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0435. \u041A\u043B\u0438\u0435\u043D\u0442\u0441\u043A\u0430\u044F \u0447\u0430\u0441\u0442\u044C \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0438\u0432\u0430\u0435\u0442 \u0434\u043E\u0441\u0442\u0443\u043F \u043A \u043F\u043E\u043A\u0430\u0437\u0430\u0442\u0435\u043B\u044F\u043C \u0437\u0430 \u0440\u0430\u0437\u043D\u044B\u0435 \u043F\u0435\u0440\u0438\u043E\u0434\u044B \u0432\u0440\u0435\u043C\u0435\u043D\u0438. \u0411\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u044D\u0442\u043E\u043C\u0443 \u0434\u043E\u0441\u0442\u0443\u043F\u0443, \u043C\u044B \u043C\u043E\u0436\u0435\u043C \u043E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u0442\u044C \u0438 \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u0438\u0440\u043E\u0432\u0430\u0442\u044C, \u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0438 \u0441\u043D\u0438\u0436\u0430\u0442\u044C \u0438\u0437\u0434\u0435\u0440\u0436\u043A\u0438 \u043D\u0430 \u0437\u0430\u0442\u0440\u0430\u0442\u044B, \u0435\u0441\u043B\u0438 \u043E\u043D\u0438 \u043F\u0440\u0438\u0441\u0443\u0442\u0441\u0442\u0432\u0443\u044E\u0442. \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u2014 \u044D\u0442\u043E \u043D\u0435 \u0442\u043E\u043B\u044C\u043A\u043E \u043C\u043D\u043E\u0436\u0435\u0441\u0442\u0432\u043E \u0434\u0430\u0442\u0447\u0438\u043A\u043E\u0432 \u0438 \u0434\u0435\u0432\u0430\u0439\u0441\u043E\u0432, \u043E\u0431\u044A\u0435\u0434\u0438\u043D\u0435\u043D\u043D\u044B\u0445 \u043C\u0435\u0436\u0434\u0443 \u0441\u043E\u0431\u043E\u0439 \u043F\u0440\u043E\u0432\u043E\u0434\u043D\u044B\u043C\u0438 \u0438 \u0431\u0435\u0441\u043F\u0440\u043E\u0432\u043E\u0434\u043D\u044B\u043C\u0438 \u0441\u0435\u0442\u044F\u043C\u0438. \u0412 \u0434\u0430\u043D\u043D\u044B\u0439 \u043C\u043E\u043C\u0435\u043D\u0442 \u044D\u0442\u043E \u0442\u0435\u0441\u043D\u0430\u044F \u0438\u043D\u0442\u0435\u0433\u0440\u0430\u0446\u0438\u044F \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u0438 \u0432\u0438\u0440\u0442\u0443\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u043C\u0438\u0440\u043E\u0432, \u0433\u0434\u0435 \u043E\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u043B\u044F\u0435\u0442\u0441\u044F \xAB\u043E\u0431\u0449\u0435\u043D\u0438\u0435\xBB \u043C\u0435\u0436\u0434\u0443 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430\u043C\u0438 \u0438 \u043B\u044E\u0434\u044C\u043C\u0438.  \u041A\u0440\u0430\u0442\u043A\u043E IoT \u2013 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044E \u043C\u043E\u0436\u043D\u043E \u043E\u0431\u044A\u044F\u0441\u043D\u0438\u0442\u044C \u0442\u0430\u043A: \u044D\u0442\u043E \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0435 \u0434\u0435\u0432\u0430\u0439\u0441\u043E\u0432 \u043F\u043E \u0441\u0445\u0435\u043C\u0435 \xAB\u043C\u0430\u0448\u0438\u043D\u0430-\u043C\u0430\u0448\u0438\u043D\u0430\xBB \u0441 \u043C\u0438\u043D\u0438\u043C\u0430\u043B\u044C\u043D\u044B\u043C \u0443\u0447\u0430\u0441\u0442\u0438\u0435\u043C \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430 \u0432 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u0435.  \u0413\u043B\u0430\u0432\u043D\u0430\u044F \u043E\u0441\u043E\u0431\u0435\u043D\u043D\u043E\u0441\u0442\u044C \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0430 \u0432\u0435\u0449\u0435\u0439\xBB  \u041E\u0441\u043E\u0431\u0435\u043D\u043D\u043E\u0441\u0442\u044C \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0441\u043E\u0441\u0442\u043E\u0438\u0442 \u0432 \u0442\u043E\u043C, \u0447\u0442\u043E \u043E\u043D\u0430 \u0443\u043F\u0440\u043E\u0449\u0430\u0435\u0442 \u0436\u0438\u0437\u043D\u044C \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430 \u043F\u043E\u0441\u0440\u0435\u0434\u0441\u0442\u0432\u043E\u043C \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u044F \u043E\u043A\u0440\u0443\u0436\u0430\u044E\u0449\u0435\u0439 \u0441\u0440\u0435\u0434\u044B \u0447\u0435\u0440\u0435\u0437 \u0446\u0438\u0444\u0440\u043E\u0432\u044B\u0435 \u0441\u0440\u0435\u0434\u0441\u0442\u0432\u0430. \u0422\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 \u0441\u043E\u0431\u043E\u0439 \u043A\u043E\u043D\u0446\u0435\u043F\u0446\u0438\u044E \u043F\u0440\u043E\u0441\u0442\u0440\u0430\u043D\u0441\u0442\u0432\u0430, \u0432 \u043A\u043E\u0442\u043E\u0440\u043E\u043C \u0434\u0435\u0432\u0430\u0439\u0441\u044B \u0441\u043E\u0432\u043C\u0435\u0449\u0435\u043D\u044B \u043C\u0435\u0436\u0434\u0443 \u0441\u043E\u0431\u043E\u0439 \u0438 \u043D\u0435 \u0442\u0440\u0435\u0431\u0443\u044E\u0442 \u0432\u043C\u0435\u0448\u0430\u0442\u0435\u043B\u044C\u0441\u0442\u0432\u0430 \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430 \u0432 \u0440\u0430\u0431\u043E\u0442\u0443 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432.  \u041F\u0440\u0438\u043C\u0435\u043D\u0435\u043D\u0438\u0435 IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0432 \u0440\u0430\u0437\u043D\u044B\u0445 \u0441\u0435\u043A\u0442\u043E\u0440\u0430\u0445 \u041E\u0431\u0449\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0439 \u0441\u0435\u043A\u0442\u043E\u0440 \u0438\u043B\u0438 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u043E \u0433\u043E\u0440\u043E\u0434\u0430 \u041E\u0431\u0449\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u044B\u0439 \u0442\u0440\u0430\u043D\u0441\u043F\u043E\u0440\u0442 \u0432 \u0433\u043E\u0440\u043E\u0434\u0435 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D \u043A \u0434\u0430\u0442\u0447\u0438\u043A\u0430\u043C \u043F\u0435\u0440\u0435\u043C\u0435\u0449\u0435\u043D\u0438\u044F, \u0447\u0435\u043B\u043E\u0432\u0435\u043A \u043C\u043E\u0436\u0435\u0442 \u043E\u0442\u0441\u043B\u0435\u0434\u0438\u0442\u044C \u0432\u0440\u0435\u043C\u044F \u043F\u0440\u0438\u0431\u044B\u0442\u0438\u044F \u043A\u043E\u043D\u043A\u0440\u0435\u0442\u043D\u043E\u0433\u043E \u0442\u0440\u0430\u043D\u0441\u043F\u043E\u0440\u0442\u043D\u043E\u0433\u043E \u0441\u0440\u0435\u0434\u0441\u0442\u0432\u0430. \u041D\u0430 \u043C\u0443\u0441\u043E\u0440\u043D\u044B\u0445 \u0431\u0430\u043A\u0430\u0445 \u0432\u0441\u0435 \u0447\u0430\u0449\u0435 \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u044E\u0442\u0441\u044F \u0434\u0430\u0442\u0447\u0438\u043A\u0438 \u043D\u0430\u043F\u043E\u043B\u043D\u0435\u043D\u0438\u044F, \u044D\u0442\u043E \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u0435\u0442 \u0441\u0432\u043E\u0435\u0432\u0440\u0435\u043C\u0435\u043D\u043D\u043E \u0443\u0432\u043E\u0437\u0438\u0442\u044C \u043C\u0443\u0441\u043E\u0440. \u0423\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0435\u0442\u0441\u044F \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u044C \u043D\u0430\u0434 \u0443\u0440\u043E\u0432\u043D\u0435\u043C \u0432\u043E\u0434\u044B \u0432 \u0432\u043E\u0434\u043E\u0435\u043C\u0430\u0445  \u0431\u0435\u0437 \u0443\u0447\u0430\u0441\u0442\u0438\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430. \u0414\u0430\u0442\u0447\u0438\u043A\u0438 \u0448\u0443\u043C\u0430 \u0438 \u0437\u0430\u0433\u0440\u044F\u0437\u043D\u0435\u043D\u0438\u044F \u0434\u0435\u043B\u0430\u044E\u0442 \u0433\u043E\u0440\u043E\u0434 \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u0435\u0435.  \u0410\u0433\u0440\u0430\u0440\u043D\u044B\u0439 \u0441\u0435\u043A\u0442\u043E\u0440 \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u043F\u043E\u0437\u0432\u043E\u043B\u0438\u043B\u0438 \u0430\u0433\u0440\u043E\u043D\u043E\u043C\u0430\u043C \u0438\u0437\u0431\u0430\u0432\u0438\u0442\u044C\u0441\u044F \u043E\u0442 \u043E\u0442\u043D\u043E\u0441\u0438\u0442\u0435\u043B\u044C\u043D\u043E\u0433\u043E \u0441\u043E\u0441\u0442\u043E\u044F\u043D\u0438\u044F \u043F\u043E\u0447\u0432. \u0423\u0441\u0442\u0430\u043D\u043E\u0432\u043B\u0435\u043D\u043D\u0430\u044F IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0432 \u0432\u0438\u0434\u0435 \u0434\u0430\u0442\u0447\u0438\u043A\u043E\u0432 \u0444\u0438\u043A\u0441\u0438\u0440\u0443\u0435\u0442 \u0440\u0430\u0437\u043B\u0438\u0447\u043D\u044B\u0435 \u043F\u043E\u043A\u0430\u0437\u0430\u0442\u0435\u043B\u0438 \u0441\u043E\u0441\u0442\u043E\u044F\u043D\u0438\u044F \u043F\u043E\u0447\u0432\u044B: \u0432\u043B\u0430\u0433\u0430, \u043F\u0438\u0442\u0430\u043D\u0438\u0435 \u0440\u0430\u0441\u0442\u0435\u043D\u0438\u0439, \u0442\u0435\u043C\u043F\u0435\u0440\u0430\u0442\u0443\u0440\u0430. \u0422\u0435\u043F\u0435\u0440\u044C \u043D\u0435\u0442 \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E\u0441\u0442\u0438 \u043E\u0431\u0445\u043E\u0434\u0438\u0442\u044C \u043F\u043E\u043B\u044F, \u0447\u0442\u043E\u0431\u044B \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u0438\u0440\u043E\u0432\u0430\u0442\u044C \u0443\u0440\u043E\u0436\u0430\u0439, \u0440\u0430\u0431\u043E\u0442\u0443 \u043E\u0442\u043D\u044B\u043D\u0435 \u043C\u043E\u0433\u0443\u0442 \u0432\u044B\u043F\u043E\u043B\u043D\u0438\u0442\u044C \u0433\u0430\u0434\u0436\u0435\u0442\u044B. \u0422\u0430\u043A, \u043D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u043D\u0435\u0431\u043E\u043B\u044C\u0448\u0430\u044F \u0441\u0442\u0440\u0430\u043D\u0430 \u041D\u0438\u0434\u0435\u0440\u043B\u0430\u043D\u0434\u044B \u0430\u043A\u0442\u0438\u0432\u043D\u043E \u043F\u0440\u0438\u043C\u0435\u043D\u044F\u044E\u0442 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044E IoT , \u0438 \u043D\u0430 \u0434\u0430\u043D\u043D\u044B\u0439 \u043C\u043E\u043C\u0435\u043D\u0442 \u044F\u0432\u043B\u044F\u0435\u0442\u0441\u044F \u043E\u0434\u043D\u043E\u0439 \u0441\u0442\u0440\u0430\u043D\u043E\u0439 \u0438\u0437 \u043C\u0438\u0440\u043E\u0432\u044B\u0445 \u043B\u0438\u0434\u0435\u0440\u043E\u0432 \u043F\u043E \u0432\u044B\u0440\u0430\u0449\u0438\u0432\u0430\u043D\u0438\u044E \u043F\u0440\u043E\u0434\u0443\u043A\u0442\u043E\u0432 \u043F\u0438\u0442\u0430\u043D\u0438\u044F, \u044D\u0442\u043E \u0441\u0442\u0430\u043B\u043E \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u044B\u043C \u0431\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u043F\u0440\u0438\u043C\u0435\u043D\u0435\u043D\u0438\u044E \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB.  \u0421\u0435\u043A\u0442\u043E\u0440 \u043B\u043E\u0433\u0438\u0441\u0442\u0438\u043A\u0438 \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0443\u043F\u0440\u043E\u0441\u0442\u0438\u043B\u0438 \u0434\u043E\u0441\u0442\u0430\u0432\u043A\u0443 \u0442\u043E\u0432\u0430\u0440\u043E\u0432 \u0441 \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0441\u0442\u0432\u0430 \u0438\u043B\u0438 \u043C\u0430\u0433\u0430\u0437\u0438\u043D\u043E\u0432 \u043A \u043F\u043E\u043A\u0443\u043F\u0430\u0442\u0435\u043B\u044E \u0431\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u043E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u043D\u0438\u044E \u043F\u043E\u0441\u044B\u043B\u043A\u0438. \u041D\u0430 \u0434\u0430\u043D\u043D\u044B\u0439 \u043C\u043E\u043C\u0435\u043D\u0442 \u0432 \u043B\u043E\u0433\u0438\u0441\u0442\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u044F\u0445 \u0430\u043A\u0442\u0438\u0432\u043D\u043E \u043F\u0440\u0438\u043C\u0435\u043D\u044F\u0435\u0442\u0441\u044F \u043E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u043D\u0438\u0435 \u0433\u0440\u0443\u0437\u043E\u0432\u044B\u0445 \u0430\u0432\u0442\u043E\u043C\u043E\u0431\u0438\u043B\u0435\u0439 \u043F\u043E \u0434\u0430\u0442\u0447\u0438\u043A\u0430\u043C, \u0438\u0445 \u0441\u043E\u0441\u0442\u043E\u044F\u043D\u0438\u0435 \u0438 \u0432\u044B\u0431\u0440\u043E\u0441\u044B \u0432 \u043E\u043A\u0440\u0443\u0436\u0430\u044E\u0449\u0443\u044E \u0441\u0440\u0435\u0434\u0443.  \u0421\u0442\u0440\u043E\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u0439 \u0441\u0435\u043A\u0442\u043E\u0440 \u041D\u0430\u0438\u0431\u043E\u043B\u044C\u0448\u0443\u044E \u043F\u043E\u043F\u0443\u043B\u044F\u0440\u043D\u043E\u0441\u0442\u044C \u0432 \u0441\u043E\u0432\u0440\u0435\u043C\u0435\u043D\u043D\u043E\u043C \u043C\u0438\u0440\u0435 \u043D\u0430\u0431\u0438\u0440\u0430\u0435\u0442 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \xAB\u0423\u043C\u043D\u044B\u0439 \u0434\u043E\u043C\xBB. \u041C\u043E\u0436\u043D\u043E \u0441\u043A\u0430\u0437\u0430\u0442\u044C, \u0447\u0442\u043E \u0443\u043C\u043D\u044B\u0435 \u0441\u0447\u0435\u0442\u0447\u0438\u043A\u0438 \u0444\u0438\u043A\u0441\u0438\u0440\u0443\u044E\u0442 \u0441\u043A\u043E\u043B\u044C\u043A\u043E \u044D\u043D\u0435\u0440\u0433\u0438\u0438 \u0438 \u0437\u0430\u0442\u0440\u0430\u0442 \u043D\u0430 \u043D\u0435\u0451 \u0431\u044B\u043B\u043E \u043F\u043E\u0442\u0440\u0430\u0447\u0435\u043D\u043E \u0432 \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u043D\u043E\u043C \u043C\u0435\u0441\u044F\u0446\u0435, \u043F\u043E\u043A\u0430\u0437\u0430\u043D\u0438\u044F \u043F\u0435\u0440\u0435\u0434\u0430\u044E\u0442\u0441\u044F \u043D\u0430 \u043E\u043F\u043B\u0430\u0442\u0443 \u0431\u0435\u0437 \u0443\u0447\u0430\u0441\u0442\u0438\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430. \u0414\u0430\u043D\u043D\u0430\u044F \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u0442 \u0448\u0438\u0440\u043E\u043A\u0438\u0439 \u0441\u043F\u0435\u043A\u0442\u0440 \u043F\u0440\u0435\u0434\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0435\u043C\u044B\u0445 \u0443\u0441\u043B\u0443\u0433.  \u0427\u0442\u043E \u0442\u0430\u043A\u043E\u0435 \xAB\u0443\u043C\u043D\u044B\u0439 \u0434\u043E\u043C\xBB \u0423\u043C\u043D\u044B\u0439 \u0434\u043E\u043C \u041F\u043E\u043D\u044F\u0442\u0438\u0435 \xAB\u0423\u043C\u043D\u044B\u0439 \u0434\u043E\u043C\xBB \u043F\u043E\u044F\u0432\u0438\u043B\u043E\u0441\u044C \u0435\u0449\u0435 \u0432 1985 \u0433\u043E\u0434\u0443, \u0442\u043E\u0433\u0434\u0430 \u0432 \u0421\u0428\u0410 \u0431\u044B\u043B\u0430 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0430\u043D\u0430 \u0441\u0438\u0441\u0442\u0435\u043C\u0430 \xABUnity\xBB, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u0443\u043F\u0440\u0430\u0432\u043B\u044F\u043B\u0430 \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u044C\u044E \u0434\u043E\u043C\u0430. \u0412 \u043D\u0430\u0441\u0442\u043E\u044F\u0449\u0435\u0435 \u0432\u0440\u0435\u043C\u044F \xABSmart Home\xBB \u043F\u043E\u0434\u0440\u0430\u0437\u0443\u043C\u0435\u0432\u0430\u0435\u0442 \u043F\u043E\u0434 \u0441\u043E\u0431\u043E\u0439 \u043D\u0435 \u0442\u043E\u043B\u044C\u043A\u043E \u0443\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u0435 \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u044C\u044E, \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0440\u0435\u0433\u0443\u043B\u0438\u0440\u0443\u0435\u0442 \u0438 \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u0438\u0440\u0443\u0435\u0442 \u0431\u043E\u043B\u0435\u0435 \u0441\u043B\u043E\u0436\u043D\u044B\u0435 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u044B. \u0412 \u043A\u043E\u043D\u0446\u0435\u043F\u0446\u0438\u044E \xAB\u0423\u043C\u043D\u044B\u0439 \u0434\u043E\u043C\xBB  \u0432\u043A\u043B\u044E\u0447\u0435\u043D\u044B \u0434\u0430\u0442\u0447\u0438\u043A\u0438 \u0438 \u0443\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0449\u0438\u0435 \u044D\u043B\u0435\u043C\u0435\u043D\u0442\u044B, \u0438\u0441\u043F\u043E\u043B\u043D\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u043D\u0430\u0441\u0442\u0440\u0430\u0438\u0432\u0430\u044E\u0442\u0441\u044F \u0438 \u043F\u0440\u043E\u0433\u0440\u0430\u043C\u043C\u0438\u0440\u0443\u044E\u0442\u0441\u044F \u043F\u043E\u0434 \u043F\u043E\u0442\u0440\u0435\u0431\u043D\u043E\u0441\u0442\u0438 \u043E\u043F\u0440\u0435\u0434\u0435\u043B\u0435\u043D\u043D\u043E\u0433\u043E \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044F. \u0423\u043F\u0440\u0430\u0432\u043B\u044F\u044E\u0449\u0438\u0435 \u044D\u043B\u0435\u043C\u0435\u043D\u0442\u044B \u043F\u043E\u043B\u0443\u0447\u0430\u044E\u0442 \u0441\u0438\u0433\u043D\u0430\u043B \u0441 \u0434\u0430\u0442\u0447\u0438\u043A\u043E\u0432 \u0438 \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u0438\u0440\u0443\u044E\u0442 \u0440\u0430\u0431\u043E\u0442\u0443 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0438\u0441\u043F\u043E\u043B\u043D\u044F\u044E\u0442 \u0437\u0430\u0434\u0430\u0447\u0443, \u0441\u043B\u0435\u0434\u0443\u044F \u0437\u0430\u0434\u0430\u043D\u043D\u044B\u043C \u0430\u043B\u0433\u043E\u0440\u0438\u0442\u043C\u0430\u043C. \u0421\u0438\u0441\u0442\u0435\u043C\u0430 \u043E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u0435\u0442 \u0438 \u0440\u0435\u0433\u0443\u043B\u0438\u0440\u0443\u0435\u0442 \u043E\u0442\u043E\u043F\u043B\u0435\u043D\u0438\u0435 \u0438 \u043E\u0445\u043B\u0430\u0436\u0434\u0435\u043D\u0438\u0435 \u0434\u043E\u043C\u0430, \u043E\u0445\u0440\u0430\u043D\u043D\u0443\u044E \u0438 \u043F\u043E\u0436\u0430\u0440\u043D\u0443\u044E \u0441\u0438\u0433\u043D\u0430\u043B\u0438\u0437\u0430\u0446\u0438\u044E, \u0432\u0438\u0434\u0435\u043E\u043D\u0430\u0431\u043B\u044E\u0434\u0435\u043D\u0438\u0435 \u0438 \u043E\u0441\u0432\u0435\u0449\u0435\u043D\u0438\u0435.  \xAB\u0423\u043C\u043D\u044B\u0439 \u0434\u043E\u043C\xBB \u043E\u0431\u044A\u0435\u0434\u0438\u043D\u044F\u0435\u0442 \u043F\u043E\u043A\u0430\u0437\u0430\u043D\u0438\u044F \xAB\u0443\u043C\u043D\u044B\u0445\xBB \u043F\u0440\u0438\u0431\u043E\u0440\u043E\u0432 \u0438 \u0442\u0440\u0430\u043D\u0441\u0444\u043E\u0440\u043C\u0438\u0440\u0443\u0435\u0442 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044E \u0432 \u0443\u0434\u043E\u0431\u043D\u043E\u043C \u0434\u043B\u044F \u043D\u0435\u0433\u043E \u0432\u0438\u0434\u0435 \u2013 \u043D\u0430 \u044D\u043A\u0440\u0430\u043D \u0442\u0435\u043B\u0435\u0432\u0438\u0437\u043E\u0440\u0430 \u0438\u043B\u0438 \u0441\u043C\u0430\u0440\u0442\u0444\u043E\u043D\u0430.  \u0413\u043B\u0430\u0432\u043D\u0430\u044F \u0437\u0430\u0434\u0430\u0447\u0430 \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0438 \u0432 \u0447\u0430\u0441\u0442\u043D\u043E\u0441\u0442\u0438 \xAB\u0423\u043C\u043D\u043E\u0433\u043E \u0434\u043E\u043C\u0430\xBB \u2014 \u044D\u0442\u043E \u0443\u043F\u0440\u043E\u0449\u0435\u043D\u0438\u0435 \u0438 \u0441\u0443\u0449\u0435\u0441\u0442\u0432\u0435\u043D\u043D\u0430\u044F \u044D\u043A\u043E\u043D\u043E\u043C\u0438\u044F \u0441\u0440\u0435\u0434\u0441\u0442\u0432 \u0438 \u0432\u0440\u0435\u043C\u0435\u043D\u0438. \u0420\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u0447\u0438\u043A\u0438  IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0442\u0430\u043A\u0436\u0435 \u0441\u0434\u0435\u043B\u0430\u043B\u0438 \u0431\u043E\u043B\u044C\u0448\u043E\u0439 \u0448\u0430\u0433 \u0432\u043F\u0435\u0440\u0435\u0434 \u0432 \u043D\u0435 \u0441\u0430\u043C\u0443\u044E \u043E\u0447\u0435\u0432\u0438\u0434\u043D\u0443\u044E, \u043D\u043E \u0432\u0430\u0436\u043D\u0443\u044E \u043E\u0431\u043B\u0430\u0441\u0442\u044C \u0437\u0434\u0440\u0430\u0432\u043E\u043E\u0445\u0440\u0430\u043D\u0435\u043D\u0438\u044F \u2013 \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0443.  \u0420\u0430\u0437\u0432\u0438\u0442\u0438\u0435 \xAB\u0443\u043C\u043D\u043E\u0439 \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u044B\xBB \u0432 \u0442\u0435\u0447\u0435\u043D\u0438\u0435 \u043F\u044F\u0442\u0438 \u043B\u0435\u0442 \u0423\u043C\u043D\u0430\u044F \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0430 \u0410\u043D\u0430\u043B\u0438\u0442\u0438\u043A\u0438 \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u0438 Markets and Markets \u0441\u0447\u0438\u0442\u0430\u044E\u0442, \u0447\u0442\u043E \u0432 \u0431\u043B\u0438\u0436\u0430\u0439\u0448\u0438\u0435 \u043F\u044F\u0442\u044C \u043B\u0435\u0442 \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0430 \u0441\u0442\u0430\u043D\u0435\u0442 \u0441\u0430\u043C\u044B\u043C \u0431\u044B\u0441\u0442\u0440\u043E\u0440\u0430\u0441\u0442\u0443\u0449\u0438\u043C \u0441\u0435\u0433\u043C\u0435\u043D\u0442\u043E\u043C \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB.  \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442-\u0432\u0435\u0449\u0435\u0439\xBB \u0432 \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0435 \u2013 \u044D\u0442\u043E \u0441\u0435\u0442\u044C \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0445 \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0441\u043A\u0438\u0445 \u0434\u0435\u0432\u0430\u0439\u0441\u043E\u0432, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u0441\u043E\u0431\u0438\u0440\u0430\u044E\u0442 \u0434\u0430\u043D\u043D\u044B\u0435 \u0438 \u043F\u0435\u0440\u0435\u0434\u0430\u044E\u0442 \u0438\u0445 \u0432 \u0418\u0422-\u0441\u0438\u0441\u0442\u0435\u043C\u0443 \u0434\u043B\u044F \u0434\u0430\u043B\u044C\u043D\u0435\u0439\u0448\u0435\u0433\u043E \u0430\u043D\u0430\u043B\u0438\u0437\u0430. \u0414\u0435\u0432\u0430\u0439\u0441\u0430\u043C\u0438 \u043C\u043E\u0433\u0443\u0442 \u0431\u044B\u0442\u044C \u043D\u043E\u0441\u0438\u043C\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430, \u043C\u043E\u0431\u0438\u043B\u044C\u043D\u044B\u0435 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F, \u0442\u0430\u0431\u043B\u0435\u0442\u043A\u0438, \u0441\u043C\u0430\u0440\u0442-\u043A\u0440\u043E\u0432\u0430\u0442\u0438 \u0438 \u0443\u043C\u043D\u044B\u0435 \u0438\u0437\u043C\u0435\u0440\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430.  \u0422\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u2014 IoT \u0432 \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0435 \u0438\u0437\u043C\u0435\u043D\u0438\u0442 \u043B\u0435\u0447\u0435\u043D\u0438\u0435 \u043F\u0430\u0446\u0438\u0435\u043D\u0442\u043E\u0432, \u043E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u043D\u0438\u0435 \u0441\u043E\u0441\u0442\u043E\u044F\u043D\u0438\u044F, \u0441\u0431\u043E\u0440 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u0438.   \u0420\u0435\u0448\u0435\u043D\u0438\u044F \u2014 IoT \u0431\u0443\u0434\u0443\u0442 \u043F\u0440\u0435\u0434\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0442\u044C \u0434\u0430\u043D\u043D\u044B\u0435 \u043F\u043E \u0442\u0435\u043A\u0443\u0449\u0435\u043C\u0443 \u0441\u043E\u0441\u0442\u043E\u044F\u043D\u0438\u044E \u043F\u0430\u0446\u0438\u0435\u043D\u0442\u0430, \u0441\u043B\u0435\u0434\u0438\u0442\u044C \u0437\u0430 \u0435\u0433\u043E \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u044F\u043C\u0438 \u0438 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u043F\u0440\u0435\u0434\u0443\u043F\u0440\u0435\u0436\u0434\u0430\u0442\u044C \u043E \u043F\u043E\u0442\u0435\u043D\u0446\u0438\u0430\u043B\u044C\u043D\u044B\u0445 \u043F\u0440\u043E\u0431\u043B\u0435\u043C\u0430\u0445 \u0434\u043E \u0442\u043E\u0433\u043E, \u043A\u0430\u043A \u043E\u043D\u0438 \u0441\u0442\u0430\u043D\u0443\u0442 \u043A\u0440\u0438\u0442\u0438\u0447\u0435\u0441\u043A\u0438\u043C\u0438. \u0410\u043D\u0430\u043B\u0438\u0437 \u0434\u0430\u043D\u043D\u044B\u0445 \u0441 \u043F\u043E\u043C\u043E\u0449\u044C\u044E \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432 \u043D\u0430\u043F\u0440\u0430\u0432\u043B\u0435\u043D \u043D\u0430 \u0443\u043B\u0443\u0447\u0448\u0435\u043D\u0438\u0435  \u0434\u0438\u0430\u0433\u043D\u043E\u0441\u0442\u0438\u043A\u0438 \u0441\u043E\u0441\u0442\u043E\u044F\u043D\u0438\u044F \u043F\u0430\u0446\u0438\u0435\u043D\u0442\u043E\u0432. \u041F\u043E\u0441\u0442\u043E\u044F\u043D\u043D\u043E\u0435 \u043D\u0430\u0431\u043B\u044E\u0434\u0435\u043D\u0438\u0435 \u0437\u0430 \u043F\u0430\u0446\u0438\u0435\u043D\u0442\u0430\u043C\u0438 \u043F\u043E\u0437\u0432\u043E\u043B\u044F\u0442 \u0432\u0440\u0430\u0447\u0430\u043C \u0438\u043C\u0435\u0442\u044C \u0446\u0435\u043B\u043E\u0441\u0442\u043D\u0443\u044E \u0438 \u0430\u043A\u0442\u0443\u0430\u043B\u044C\u043D\u0443\u044E \u043A\u0430\u0440\u0442\u0438\u043D\u0443.   \u0414\u0430\u0442\u0447\u0438\u043A\u0438 \u0438 \u0443\u043C\u043D\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0441 \u043C\u043E\u0431\u0438\u043B\u044C\u043D\u044B\u043C\u0438 \u0438 \u0432\u0435\u0431-\u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u043C\u0438 \u043F\u0440\u0438\u0432\u0435\u0434\u0443\u0442 \u043A \u0443\u043C\u0435\u043D\u044C\u0448\u0435\u043D\u0438\u044E \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043D\u0438\u044F \u044D\u043D\u0435\u0440\u0433\u043E\u0435\u043C\u043A\u043E\u0433\u043E \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0441\u043A\u043E\u0433\u043E \u043E\u0431\u043E\u0440\u0443\u0434\u043E\u0432\u0430\u043D\u0438\u044F \u0438 \u0441\u043E\u043A\u0440\u0430\u0449\u0435\u043D\u0438\u044E \u043E\u043F\u0435\u0440\u0430\u0446\u0438\u043E\u043D\u043D\u044B\u0445 \u0440\u0430\u0441\u0445\u043E\u0434\u043E\u0432.  \u0423\u043C\u043D\u044B\u0435 \u0442\u0430\u0431\u043B\u0435\u0442\u043A\u0438, \u0431\u0435\u0441\u043F\u0440\u043E\u0432\u043E\u0434\u043D\u044B\u0435 \u043F\u043B\u0430\u0441\u0442\u044B\u0440\u0438, \u043A\u043E\u0440\u043E\u0431\u043A\u0430 \u0434\u043B\u044F \u043B\u0435\u043A\u0430\u0440\u0441\u0442\u0432 \u0441 \u043D\u0430\u043F\u043E\u043C\u0438\u043D\u0430\u043D\u0438\u0435\u043C \u043E \u0432\u0440\u0435\u043C\u0435\u043D\u0438 \u0443\u043F\u043E\u0442\u0440\u0435\u0431\u043B\u0435\u043D\u0438\u044F \u043B\u0435\u043A\u0430\u0440\u0441\u0442\u0432 \u0438 \u0433\u0440\u043E\u043C\u043A\u043E\u0433\u043E \u0441\u0438\u0433\u043D\u0430\u043B\u0430 \u0432 \u0441\u043B\u0443\u0447\u0430\u0435 \u0437\u0430\u0431\u044B\u0442\u043E\u0439 \u0442\u0430\u0431\u043B\u0435\u0442\u043A\u0438 \u2013 \u043F\u0440\u0438\u043C\u0435\u0440\u044B \u043C\u0435\u0434\u0438\u0446\u0438\u043D\u0441\u043A\u0438\u0445 IoT-\u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432.  \xAB\u0423\u043C\u043D\u044B\u0439 \u0440\u0438\u0442\u0435\u0439\u043B\xBB \u0438\u043B\u0438 \u043A\u0430\u043A \u0431\u0443\u0434\u0443\u0442 \u0432\u044B\u0433\u043B\u044F\u0434\u0435\u0442\u044C \u043C\u0430\u0433\u0430\u0437\u0438\u043D\u044B \u0431\u0443\u0434\u0443\u0449\u0435\u0433\u043E \u0423\u043C\u043D\u044B\u0439 \u0440\u0438\u0442\u0435\u0439\u043B \u0412 \u0431\u0443\u0434\u0443\u0449\u0435\u043C \u0434\u0430\u0442\u0447\u0438\u043A\u0438, \u0440\u0430\u0434\u0438\u043E\u043C\u0430\u044F\u043A\u0438 \u0438 \u0434\u0440\u0443\u0433\u0438\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 IoT \u0440\u0435\u0430\u043B\u0438\u0437\u0443\u044E\u0442 \u0438\u043D\u0434\u0438\u0432\u0438\u0434\u0443\u0430\u043B\u044C\u043D\u044B\u0439 \u043F\u043E\u0434\u0445\u043E\u0434 \u043A \u043A\u0430\u0436\u0434\u043E\u043C\u0443 \u043F\u043E\u043A\u0443\u043F\u0430\u0442\u0435\u043B\u044E. \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB  \u043F\u043E\u0434\u0441\u043A\u0430\u0436\u0435\u0442, \u0433\u0434\u0435 \u043D\u0430\u0445\u043E\u0434\u0438\u0442\u0441\u044F \xAB\u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u043D\u044B\u0439\xBB \u043F\u043E\u043A\u0443\u043F\u0430\u0442\u0435\u043B\u044C \u0438 \u043A\u0430\u043A \u0435\u043C\u0443 \u043D\u0430\u0439\u0442\u0438 \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u044B\u0439 \u043F\u0440\u043E\u0434\u0443\u043A\u0442.  \u0422\u0430\u043A \u043F\u0440\u043E\u0434\u0430\u0432\u0435\u0446 \u0432\u043B\u0430\u0434\u0435\u0435\u0442 \u043F\u043E\u043B\u043D\u043E\u0439 \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u0438 \u043E \u043F\u043E\u0442\u0435\u043D\u0446\u0438\u0430\u043B\u044C\u043D\u043E\u043C \u043F\u043E\u043A\u0443\u043F\u0430\u0442\u0435\u043B\u0435: \u0447\u0442\u043E \u0435\u0441\u0442, \u043A\u0430\u043A\u043E\u0439 \u0440\u0430\u0437\u043C\u0435\u0440 \u043E\u0434\u0435\u0436\u0434\u044B, \u043A\u0430\u043A\u0438\u043C \u0431\u0440\u0435\u043D\u0434\u0430\u043C \u043E\u0442\u0434\u0430\u0435\u0442 \u043F\u0440\u0435\u0434\u043F\u043E\u0447\u0442\u0435\u043D\u0438\u0435. \u0422\u0435\u043C \u0441\u0430\u043C\u044B\u043C, \u043E\u043D \u0432 \u043C\u043E\u043C\u0435\u043D\u0442\u0435 \u043C\u043E\u0436\u0435\u0442 \u043E\u0442\u043F\u0440\u0430\u0432\u0438\u0442\u044C \u0443\u0432\u0435\u0434\u043E\u043C\u043B\u0435\u043D\u0438\u0435 \u0447\u0435\u0440\u0435\u0437 \u043F\u0440\u0438\u043B\u043E\u0436\u0435\u043D\u0438\u0435 \u043E \u043D\u043E\u0432\u044B\u0445 \u043F\u0440\u043E\u0434\u0443\u043A\u0442\u0430\u0445, \u0442\u043E\u0432\u0430\u0440\u0430\u0445, \u0430\u043A\u0446\u0438\u044F\u0445 \u0438 \u0441\u043F\u0435\u0446\u043F\u0440\u0435\u0434\u043B\u043E\u0436\u0435\u043D\u0438\u044F\u0445 \u0442\u043E\u0433\u0434\u0430, \u043A\u043E\u0433\u0434\u0430 \u043F\u043E\u0442\u0435\u043D\u0446\u0438\u0430\u043B\u044C\u043D\u044B\u0439 \u043F\u043E\u043A\u0443\u043F\u0430\u0442\u0435\u043B\u044C \u043F\u0435\u0440\u0435\u0441\u0442\u0443\u043F\u0430\u0435\u0442 \u043F\u043E\u0440\u043E\u0433 \u043C\u0430\u0433\u0430\u0437\u0438\u043D\u0430.  \u0410\u043D\u0430\u043B\u0438\u0442\u0438\u043A\u0430 \u0432 \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u043C \u0432\u0440\u0435\u043C\u0435\u043D\u0438. IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0431\u0443\u0434\u0435\u0442 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0438\u0432\u0430\u0442\u044C \u0430\u043D\u0430\u043B\u0438\u0437 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u0430 \u043F\u0440\u043E\u0434\u0430\u043D\u043D\u044B\u0445 \u0435\u0434\u0438\u043D\u0438\u0446, \u043F\u0440\u043E\u0441\u043B\u0435\u0434\u0438\u0442 \u0437\u0430 \u043B\u0438\u0434\u0438\u0440\u0443\u044E\u0449\u0438\u043C\u0438 \u0442\u043E\u0432\u0430\u0440\u0430\u043C\u0438. \u0415\u0441\u043B\u0438 \u0433\u043B\u0430\u0432\u0435\u043D\u0441\u0442\u0432\u0443\u044E\u0449\u0438\u0439 \u0444\u0430\u043A\u0442\u043E\u0440 \u043F\u0440\u0438 \u0432\u044B\u0431\u043E\u0440\u0435 \u0442\u043E\u0432\u0430\u0440\u0430 \u2014  \u0446\u0435\u043D\u0430, \u0442\u043E \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u0441\u043A\u043E\u0440\u0440\u0435\u043A\u0442\u0438\u0440\u0443\u0435\u0442 \u0446\u0435\u043D\u0443 \u0432 \u0441\u0442\u043E\u0440\u043E\u043D\u0443 \u043F\u043E\u043D\u0438\u0436\u0435\u043D\u0438\u044F \u0438\u043B\u0438 \u043F\u043E\u0432\u044B\u0448\u0435\u043D\u0438\u044F, \u0447\u0442\u043E\u0431\u044B \u0434\u043E\u0441\u0442\u0438\u0447\u044C \u043C\u0430\u043A\u0441\u0438\u043C\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u043E\u0431\u044A\u0435\u043C\u0430 \u043F\u0440\u043E\u0434\u0430\u0436.  \u0423\u0436\u0435 \u0432 \u0431\u043B\u0438\u0436\u0430\u0439\u0448\u0438\u0435 \u043F\u044F\u0442\u044C \u043B\u0435\u0442, \u043C\u0430\u0433\u0430\u0437\u0438\u043D\u044B \u0438 \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0441\u0442\u0432\u0430 \u0431\u0443\u0434\u0443\u0442 \u043F\u0440\u0438\u043C\u0435\u043D\u044F\u0442\u044C IoT \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044E \u0432 \u0441\u0432\u043E\u0435\u0439 \u0440\u0430\u0431\u043E\u0442\u0435. \u0414\u0430\u043D\u043D\u0430\u044F \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u043A\u043E\u0441\u043D\u0435\u0442\u0441\u044F \u0442\u043E\u0432\u0430\u0440\u043D\u044B\u0445 \u0437\u0430\u043F\u0430\u0441\u043E\u0432 \u043D\u0430 \u0441\u043A\u043B\u0430\u0434\u0435. \u0422\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \u043F\u043E\u0437\u0432\u043E\u043B\u0438\u0442 \u043E\u0442\u0441\u043B\u0435\u0436\u0438\u0432\u0430\u0442\u044C \u0437\u0430\u043A\u0443\u043F\u043A\u0443, \u043F\u0440\u0438\u0432\u043E\u0437, \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0438\u0442 \u043E\u0442\u0441\u0443\u0442\u0441\u0442\u0432\u0438\u0435 \u043F\u0435\u0440\u0435\u0438\u0437\u0431\u044B\u0442\u043A\u0430 \u0442\u043E\u0432\u0430\u0440\u0430, \u043A\u043E\u0442\u043E\u0440\u044B\u0439 \u043D\u0435 \u043F\u043E\u043B\u044C\u0437\u0443\u0435\u0442\u0441\u044F \u0441\u043F\u0440\u043E\u0441\u043E\u043C, \u043D\u0430\u043B\u0430\u0434\u0438\u0442 \u0441\u0438\u0441\u0442\u0435\u043C\u0443 \u043F\u043E\u0441\u0442\u0430\u0432\u043E\u043A.IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0441\u043C\u043E\u0433\u0443\u0442 \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0438\u0442\u044C \u043A\u043E\u043C\u043F\u0430\u043D\u0438\u044F\u043C \u0438 \u043C\u0430\u0433\u0430\u0437\u0438\u043D\u0430\u043C \u0441\u043D\u0438\u0436\u0435\u043D\u0438\u0435 \u043F\u0435\u0440\u0435\u043C\u0435\u043D\u043D\u044B\u0445 \u0438 \u043F\u043E\u0441\u0442\u043E\u044F\u043D\u043D\u044B\u0445 \u0438\u0437\u0434\u0435\u0440\u0436\u0435\u043A, \u0441\u043A\u043E\u0440\u0440\u0435\u043A\u0442\u0438\u0440\u0443\u0435\u0442 \u0441\u043F\u0440\u043E\u0441 \u0438 \u043F\u0440\u0435\u0434\u043B\u043E\u0436\u0435\u043D\u0438\u0435, \u0430 \u0441\u0430\u043C\u043E\u0435 \u0433\u043B\u0430\u0432\u043D\u043E\u0435, \u0447\u0442\u043E \u0434\u0430\u0436\u0435 \u043D\u0435 \u043F\u0440\u0438\u0434\u0435\u0442\u0441\u044F \u0442\u0440\u0430\u0442\u0438\u0442\u044C\u0441\u044F \u043D\u0430 \u0434\u043E\u043F\u043E\u043B\u043D\u0438\u0442\u0435\u043B\u044C\u043D\u044B\u0439 \u043F\u0435\u0440\u0441\u043E\u043D\u0430\u043B.  \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u0432 \u0431\u043B\u0438\u0436\u0430\u0439\u0448\u0435\u0435 \u0432\u0440\u0435\u043C\u044F \u043F\u0435\u0440\u0435\u0432\u0435\u0440\u043D\u0435\u0442 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u0435\u043D\u0438\u0435 \u043B\u044E\u0434\u0435\u0439 \u043E \u043F\u043E\u043A\u0443\u043F\u043A\u0430\u0445, \u0443\u043F\u0440\u043E\u0441\u0442\u0438\u0442 \u044D\u0442\u043E\u0442 \u043F\u0440\u043E\u0446\u0435\u0441\u0441, \u0431\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0437\u0430\u0446\u0438\u0438.  \u041D\u043E\u0441\u0438\u043C\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0423\u043C\u043D\u044B\u0435 \u0447\u0430\u0441\u044B \u041E\u0433\u0440\u043E\u043C\u043D\u0443\u044E \u043F\u043E\u043F\u0443\u043B\u044F\u0440\u043D\u043E\u0441\u0442\u044C \u043F\u043E\u043B\u0443\u0447\u0438\u043B\u0438 \xAB\u0443\u043C\u043D\u044B\u0435\xBB \u043F\u043E\u043C\u043E\u0449\u043D\u0438\u043A\u0438 \u0432 \u0432\u0438\u0434\u0435 \u043D\u043E\u0441\u0438\u043C\u044B\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432 \u043D\u0430 \u0440\u0443\u043A\u0435. \u042D\u0442\u043E \u0434\u0435\u0432\u0430\u0439\u0441\u044B \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB, \u043A\u043E\u0442\u043E\u0440\u044B\u0435 \u044F\u0432\u043B\u044F\u044E\u0442\u0441\u044F \u0438\u0437\u043C\u0435\u0440\u0438\u0442\u0435\u043B\u044F\u043C\u0438 \u0444\u0438\u0437\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u043F\u043E\u043A\u0430\u0437\u0430\u0442\u0435\u043B\u0435\u0439 \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430. \xAB\u0423\u043C\u043D\u044B\u0435\xBB \u0431\u0440\u0430\u0441\u043B\u0435\u0442\u044B \u044D\u0442\u043E \u043F\u043E\u043B\u043D\u043E\u0446\u0435\u043D\u043D\u044B\u0435 \u0433\u0430\u0434\u0436\u0435\u0442\u044B \u0434\u043B\u044F \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u044F \u0437\u0434\u043E\u0440\u043E\u0432\u044C\u044F. \u0423\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0442\u0430\u043A\u0436\u0435 \u043F\u043E\u0434\u0441\u0447\u0438\u0442\u044B\u0432\u0430\u044E\u0442 \u0448\u0430\u0433\u0438, \u0430\u043D\u0430\u043B\u0438\u0437\u0438\u0440\u0443\u044E\u0442 \u0430\u043A\u0442\u0438\u0432\u043D\u043E\u0441\u0442\u044C \u0438 \u043F\u0440\u0435\u0434\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u044E\u0442 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044E \u043E\u0442\u0447\u0435\u0442. \u0410\u043D\u0430\u043B\u0438\u0442\u0438\u043A\u0438 \u043F\u043E\u0434\u0441\u0447\u0438\u0442\u0430\u043B\u0438, \u0447\u0442\u043E \u0432 2025 \u0433\u043E\u0434\u0443 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0433\u0430\u0434\u0436\u0435\u0442\u043E\u0432 \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0431\u0443\u0434\u0435\u0442 \u0441\u043E\u0441\u0442\u0430\u0432\u043B\u044F\u0442\u044C \u043E\u043A\u043E\u043B\u043E 27 \u043C\u043B\u0440\u0434 \u0435\u0434\u0438\u043D\u0438\u0446 \u2013 \u043F\u0440\u043E\u0442\u0438\u0432 8,5 \u043C\u043B\u0440\u0434 \u0441\u0435\u0433\u043E\u0434\u043D\u044F.  \u0412 \u0431\u0443\u0434\u0443\u0449\u0435\u043C \u043F\u043B\u0430\u043D\u0438\u0440\u0443\u0435\u0442\u0441\u044F \u043C\u0430\u0441\u0448\u0442\u0430\u0431\u043D\u0430\u044F \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u043A\u0430 \u0440\u0430\u0437\u043B\u0438\u0447\u043D\u044B\u0445 \u0431\u0440\u0430\u0441\u043B\u0435\u0442\u043E\u0432. \u041D\u0430\u043F\u0440\u0438\u043C\u0435\u0440, \u0431\u0440\u0430\u0441\u043B\u0435\u0442\u044B, \u043E\u043A\u0430\u0437\u044B\u0432\u0430\u044E\u0449\u0438\u0435 \u044D\u043A\u0441\u0442\u0440\u0435\u043D\u043D\u0443\u044E \u043F\u043E\u043C\u043E\u0449\u044C \u0441\u043E\u043B\u0434\u0430\u0442\u0430\u043C, \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u0438\u0440\u0443\u044E\u0449\u0438\u0435 \u0432\u043E\u0437\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0435 \u0441\u043E\u043B\u043D\u0435\u0447\u043D\u044B\u0445 \u043B\u0443\u0447\u0435\u0439 \u043D\u0430 \u043A\u043E\u0436\u0443 \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430. \u0418\u0434\u0435\u044F \u043F\u0430\u0440\u043D\u044B\u0445 \u0431\u0440\u0430\u0441\u043B\u0435\u0442\u043E\u0432 \u043D\u0430\u0431\u0440\u0430\u043B\u0430 \u043D\u0430\u0438\u0431\u043E\u043B\u044C\u0448\u0443\u044E \u043F\u043E\u043F\u0443\u043B\u044F\u0440\u043D\u043E\u0441\u0442\u044C, \u0442\u0430\u043A \u043A\u0430\u043A \u043F\u0440\u0438\u043D\u0446\u0438\u043F \u0440\u0430\u0431\u043E\u0442\u044B \u0441\u0442\u0430\u043B \u043F\u0440\u0438\u0432\u043B\u0435\u043A\u0430\u0442\u0435\u043B\u0435\u043D \u0434\u043B\u044F \u043C\u043D\u043E\u0433\u0438\u0445 \u0432\u043B\u044E\u0431\u043B\u0435\u043D\u043D\u044B\u0445: \u043F\u0440\u0438 \u043A\u0430\u0441\u0430\u043D\u0438\u0438 \u043E\u0434\u043D\u043E\u0433\u043E \u0431\u0440\u0430\u0441\u043B\u0435\u0442\u0430, \u043F\u0435\u0440\u0435\u0434\u0430\u0435\u0442\u0441\u044F \u0441\u0438\u0433\u043D\u0430\u043B \u0447\u0435\u0440\u0435\u0437 \u0440\u0430\u0441\u0441\u0442\u043E\u044F\u043D\u0438\u0435 \u043A \u0434\u0440\u0443\u0433\u043E\u043C\u0443 \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0443. \u0412\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0435 \u0447\u0435\u0440\u0435\u0437 \u0440\u0430\u0441\u0441\u0442\u043E\u044F\u043D\u0438\u0435 \u2013 \u043E\u043A\u0430\u0437\u0430\u043B\u043E\u0441\u044C \u0432\u0434\u043E\u0445\u043D\u043E\u0432\u043B\u044F\u044E\u0449\u0435\u0439 IoT- \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0435\u0439. \u041D\u043E\u0441\u0438\u043C\u044B\u0435 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u0431\u0443\u0434\u0443\u0449\u0435\u0433\u043E \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0430\u0442 \u0432\u0438\u0437\u0443\u0430\u043B\u044C\u043D\u044B\u0439 \u043A\u043E\u043D\u0442\u0430\u043A\u0442 \u0441 \u0432\u0440\u0430\u0447\u043E\u043C, \u043F\u0440\u0438\u044F\u0442\u0435\u043B\u0435\u043C \u0438\u043B\u0438 \u043F\u0430\u0440\u0442\u043D\u0435\u0440\u043E\u043C.  \u0423\u043C\u043D\u044B\u0435 \u043C\u0430\u0448\u0438\u043D\u044B \u0423\u043C\u043D\u044B\u0435 \u043C\u0430\u0448\u0438\u043D\u044B \u041C\u043D\u043E\u0433\u0438\u043C \u043B\u044E\u0434\u044F\u043C \u0443\u0436\u0435 \u0434\u043E\u0432\u0435\u043B\u043E\u0441\u044C \u0441\u0435\u0441\u0442\u044C \u0437\u0430 \u0440\u0443\u043B\u044C \xAB\u0443\u043C\u043D\u043E\u0439 \u043C\u0430\u0448\u0438\u043D\u044B\xBB, \u0438 \u0431\u043E\u043B\u044C\u0448\u0438\u043D\u0441\u0442\u0432\u043E \u0438\u0437 \u043D\u0438\u0445 \u0441\u0447\u0438\u0442\u0430\u044E\u0442, \u0447\u0442\u043E \u044D\u0442\u043E \u043E\u0434\u0438\u043D \u0438\u0437 \u043B\u0443\u0447\u0448\u0438\u0445 \u0432\u0430\u0440\u0438\u0430\u043D\u0442\u043E\u0432 \u043F\u0440\u0438\u043C\u0435\u043D\u0435\u043D\u0438\u044F IoT. \u0412 \u0434\u0430\u043D\u043D\u044B\u0439 \u043C\u043E\u043C\u0435\u043D\u0442 \u043C\u044B \u043C\u043E\u0436\u0435\u043C \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C \u0441\u0435\u043D\u0441\u043E\u0440\u044B \u0438 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u0435 \u043A \u0438\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443,  \u043F\u0440\u043E\u0433\u0440\u0435\u0432\u0430\u043D\u0438\u0435 \u0430\u0432\u0442\u043E\u043C\u043E\u0431\u0438\u043B\u044F \u0437\u0430\u0440\u0430\u043D\u0435\u0435. \u0412\u0441\u0451 \u044D\u0442\u043E \u0433\u0430\u0440\u0430\u043D\u0442\u0438\u0440\u0443\u0435\u0442 \u043D\u0430\u043C \u043C\u0430\u043A\u0441\u0438\u043C\u0430\u043B\u044C\u043D\u044B\u0439 \u043A\u043E\u043C\u0444\u043E\u0440\u0442.  \u0422\u0430\u043A\u0438\u043C \u043E\u0431\u0440\u0430\u0437\u043E\u043C, \u0432\u0441\u043A\u043E\u0440\u0435 \u0431\u0435\u0441\u043F\u0438\u043B\u043E\u0442\u043D\u044B\u0435 \u0430\u0432\u0442\u043E\u043C\u043E\u0431\u0438\u043B\u0438 \u0438\u0437\u043C\u0435\u043D\u044F\u0442 \u043D\u0430\u0448\u0435 \u043F\u0440\u0435\u0434\u0441\u0442\u0430\u0432\u043B\u0435\u043D\u0438\u0435 \u043E \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0441\u0442\u0438. \u041C\u044B \u0443\u0432\u0438\u0434\u0438\u043C \u0442\u0430\u043A\u0441\u0438 \u043F\u043E\u0434 \u0443\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u0438\u0435\u043C \u0438\u0441\u043A\u0443\u0441\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u0433\u043E \u0438\u043D\u0442\u0435\u043B\u043B\u0435\u043A\u0442\u0430.   \xAB\u0423\u043C\u043D\u044B\u0435 \u043C\u0430\u0448\u0438\u043D\u044B\xBB \u0431\u0443\u0434\u0443\u0442 \u0441\u0430\u043C\u043E\u0441\u0442\u043E\u044F\u0442\u0435\u043B\u044C\u043D\u043E \u043E\u0446\u0435\u043D\u0438\u0432\u0430\u0442\u044C \u0441\u0438\u0442\u0443\u0430\u0446\u0438\u044E \u043D\u0430 \u0434\u043E\u0440\u043E\u0433\u0435, \u0441\u043A\u043E\u0440\u0440\u0435\u043A\u0442\u0438\u0440\u0443\u044E\u0442 \u043C\u0430\u0440\u0448\u0440\u0443\u0442 \u0431\u0435\u0437 \u0443\u0447\u0430\u0441\u0442\u0438\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430. \u0421 \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u0435\u043C IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0430\u0432\u0442\u043E\u043C\u043E\u0431\u0438\u043B\u0438 \u043D\u0435 \u0441\u043C\u043E\u0433\u0443\u0442 \u043E\u0431\u043E\u0439\u0442\u0438\u0441\u044C \u0431\u0435\u0437 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0435\u043D\u0438\u044F \u043A \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442\u0443. \u0421 \u0435\u0433\u043E \u043F\u043E\u043C\u043E\u0449\u044C\u044E \u0431\u0443\u0434\u0435\u0442 \u043E\u0441\u0443\u0449\u0435\u0441\u0442\u0432\u043B\u044F\u0442\u044C\u0441\u044F \u043E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u0435 \u0442\u0440\u0430\u043D\u0441\u043F\u043E\u0440\u0442\u043D\u044B\u0445 \u043A\u0430\u0440\u0442 \u0438 \u043E\u0446\u0435\u043D\u043A\u0430 \u0434\u043E\u0440\u043E\u0436\u043D\u043E\u0439 \u0441\u0438\u0442\u0443\u0430\u0446\u0438\u0438.  \xABInternet of things\xBB \u2014 \u043E\u0436\u0438\u0434\u0430\u043D\u0438\u044F \u0438 \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0441\u0442\u044C \u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439 \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0441\u0442\u044C \u0417\u0430 \u043F\u043E\u0441\u043B\u0435\u0434\u043D\u0438\u0435 50 \u043B\u0435\u0442 \u043F\u0440\u043E\u0438\u0437\u043E\u0448\u0435\u043B \u0440\u043E\u0441\u0442 \u0432 \u043E\u0431\u043B\u0430\u0441\u0442\u0438 \u0438\u0441\u043A\u0443\u0441\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u0433\u043E \u0438\u043D\u0442\u0435\u043B\u043B\u0435\u043A\u0442\u0430.   \u0413\u0430\u0434\u0436\u0435\u0442\u044B \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u043D\u0430\u0447\u0430\u043B\u0438 \u043A\u043E\u0440\u0435\u043D\u043D\u044B\u043C \u043E\u0431\u0440\u0430\u0437\u043E\u043C \u043C\u0435\u043D\u044F\u0442\u044C \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0435 \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430 \u0441 \u043C\u0438\u0440\u043E\u043C. \u0420\u0430\u0437\u0432\u0438\u0442\u0438\u0435 \u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u043F\u0440\u0438\u0432\u0435\u0434\u0435\u0442 \u043A \u0433\u043B\u043E\u0431\u0430\u043B\u044C\u043D\u043E\u0439 \u0442\u0435\u043B\u0435\u043A\u043E\u043C\u043C\u0443\u043D\u0438\u043A\u0430\u0446\u0438\u043E\u043D\u043D\u043E\u0439 \u0438\u043D\u0444\u0440\u0430\u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u0435, \u043A\u043E\u0442\u043E\u0440\u0430\u044F \u043D\u0430\u043B\u0430\u0434\u0438\u0442 \u0432\u044B\u043F\u043E\u043B\u043D\u0435\u043D\u0438\u0435 \u043C\u043D\u043E\u0433\u0438\u0445 \u043F\u0440\u043E\u0446\u0435\u0441\u0441\u043E\u0432 \u0432 \u0430\u0432\u0442\u043E\u043C\u0430\u0442\u0438\u0447\u0435\u0441\u043A\u043E\u043C \u0440\u0435\u0436\u0438\u043C\u0435. \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0432\u043E\u0437\u044C\u043C\u0435\u0442 \u043D\u0430 \u0441\u0435\u0431\u044F \u0431\u044B\u0442\u043E\u0432\u044B\u0435 \u043F\u0440\u043E\u0431\u043B\u0435\u043C\u044B \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430, \u0447\u0442\u043E\u0431\u044B \u0442\u043E\u0442, \u0432 \u0441\u0432\u043E\u044E \u043E\u0447\u0435\u0440\u0435\u0434\u044C, \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u043B \u0432\u0440\u0435\u043C\u044F \u0431\u043E\u043B\u0435\u0435 \u044D\u0444\u0444\u0435\u043A\u0442\u0438\u0432\u043D\u043E. \u0423\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430 \u043F\u043E\u043C\u043E\u0433\u0443\u0442 \u0447\u0435\u043B\u043E\u0432\u0435\u0447\u0435\u0441\u0442\u0432\u0443 \u043E\u043F\u0442\u0438\u043C\u0430\u043B\u044C\u043D\u043E \u0440\u0430\u0441\u043F\u043E\u0440\u044F\u0436\u0430\u0442\u044C\u0441\u044F \u0440\u0435\u0441\u0443\u0440\u0441\u0430\u043C\u0438 \u2013 \u0442\u0435\u043F\u043B\u043E\u043C, \u0432\u043E\u0434\u043E\u0439 \u0438 \u044D\u043B\u0435\u043A\u0442\u0440\u043E\u044D\u043D\u0435\u0440\u0433\u0438\u0435\u0439.  \u0412 \u0434\u0430\u043D\u043D\u044B\u0439 \u043C\u043E\u043C\u0435\u043D\u0442 \u0440\u044B\u043D\u043E\u043A \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0441\u0442\u0440\u0435\u043C\u0438\u0442\u0435\u043B\u044C\u043D\u043E \u043D\u0430\u0431\u0438\u0440\u0430\u0435\u0442 \u043E\u0431\u043E\u0440\u043E\u0442\u044B, \u0437\u0430 \u0441\u0447\u0435\u0442 \u0443\u0432\u0435\u043B\u0438\u0447\u0435\u043D\u0438\u044F \u043E\u0431\u044A\u0435\u043C\u043E\u0432 \u0432\u044B\u043F\u0443\u0441\u043A\u0430 \u043D\u043E\u0432\u044B\u0445 \u0434\u0435\u0432\u0430\u0439\u0441\u043E\u0432. \u0421\u043A\u043E\u0440\u0435\u0435 \u0432\u0441\u0435\u0433\u043E, \u0432 2024 \u0433\u043E\u0434\u0443 \u0434\u0430\u0442\u0447\u0438\u043A\u0438 \u0431\u0443\u0434\u0443\u0442 \u0443\u0441\u0442\u0430\u043D\u043E\u0432\u043B\u0435\u043D\u044B \u043F\u0440\u0430\u043A\u0442\u0438\u0447\u0435\u0441\u043A\u0438 \u043D\u0430 \u0432\u0441\u0435\u0445 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u0430\u0445 \u0434\u043B\u044F \u0432\u044B\u043F\u043E\u043B\u043D\u0435\u043D\u0438\u044F \u0437\u0430\u0434\u0430\u0447 \u0443\u0441\u0442\u0440\u043E\u0439\u0441\u0442\u0432\u043E\u043C \u0431\u0435\u0437 \u0443\u0447\u0430\u0441\u0442\u0438\u044F \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430.  \u041A\u043E\u043C\u043F\u0430\u043D\u0438\u0438 \u0438 \u0441\u043E\u0432\u0440\u0435\u043C\u0435\u043D\u043D\u043E\u0435 \u043E\u0431\u0449\u0435\u0441\u0442\u0432\u043E \u043F\u0440\u043E\u0434\u043E\u043B\u0436\u0430\u044E\u0442 \u043F\u0440\u043E\u044F\u0432\u043B\u044F\u0442\u044C \u0438\u043D\u0442\u0435\u0440\u0435\u0441 \u043A \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB , \u043E\u043D\u0438 \u0432\u043E\u0437\u043B\u0430\u0433\u0430\u044E\u0442 \u0431\u043E\u043B\u044C\u0448\u0443\u044E \u043D\u0430\u0434\u0435\u0436\u0434\u0443 \u043D\u0430 \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u0435 IoT-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0438 \u043F\u0440\u043E\u0446\u0435\u043D\u0442 \u043E\u0442 \u0438\u043D\u0432\u0435\u0441\u0442\u0438\u0446\u0438\u0439 \u0432 \u043D\u0430\u0441\u0442\u043E\u044F\u0449\u0438\u0445 \u0443\u0441\u043B\u043E\u0432\u0438\u044F\u0445. \u0422\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u044F \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u0438\u0437\u043C\u0435\u043D\u0438\u0442 \u043C\u0438\u0440 \u0431\u043B\u0430\u0433\u043E\u0434\u0430\u0440\u044F \u0441\u043F\u043B\u043E\u0447\u0435\u043D\u043D\u043E\u043C\u0443 \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u0438\u044E \u0432\u0441\u0435\u0445 \u0443\u0447\u0430\u0441\u0442\u043D\u0438\u043A\u043E\u0432 \u2013 \u043E\u0442 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044F \u0434\u043E \u043F\u0440\u0430\u0432\u0438\u0442\u0435\u043B\u044C\u0441\u0442\u0432\u0430.  \u0421 \u0440\u0430\u0437\u0432\u0438\u0442\u0438\u0435\u043C Iot-\u0442\u0435\u0445\u043D\u043E\u043B\u043E\u0433\u0438\u0438 \u0432\u0441\u0435 \u0431\u043E\u043B\u044C\u0448\u0435 \u0444\u0438\u0437\u0438\u0447\u0435\u0441\u043A\u0438\u0445 \u043F\u0440\u0435\u0434\u043C\u0435\u0442\u043E\u0432 \u0431\u0443\u0434\u0443\u0442 \u043F\u043E\u0434\u043A\u043B\u044E\u0447\u0430\u0442\u044C\u0441\u044F \u043A \u0441\u0435\u0442\u0438 \u0438 \u0432\u0437\u0430\u0438\u043C\u043E\u0434\u0435\u0439\u0441\u0442\u0432\u043E\u0432\u0430\u0442\u044C \u0434\u0440\u0443\u0433 \u0441 \u0434\u0440\u0443\u0433\u043E\u043C. IoT \u043F\u0440\u0435\u0432\u0440\u0430\u0442\u0438\u0442\u0441\u044F \u0432 \u043E\u0441\u043E\u0431\u0435\u043D\u043D\u0443\u044E \u0438\u043D\u0444\u0440\u0430\u0441\u0442\u0440\u0443\u043A\u0442\u0443\u0440\u0443 \u0441\u043E\u0432\u0440\u0435\u043C\u0435\u043D\u043D\u043E\u0433\u043E \u0447\u0435\u043B\u043E\u0432\u0435\u043A\u0430, \u0433\u0434\u0435 \u043A\u0430\u0436\u0434\u043E\u0435 \u0434\u0435\u0439\u0441\u0442\u0432\u0438\u0435 \u0431\u0443\u0434\u0435\u0442 \u043F\u043E\u0434\u0447\u0438\u043D\u0435\u043D\u043E \u0438\u0441\u043A\u0443\u0441\u0441\u0442\u0432\u0435\u043D\u043D\u043E\u043C\u0443 \u0438\u043D\u0442\u0435\u043B\u043B\u0435\u043A\u0442\u0443. \u0412 \u0431\u043B\u0438\u0436\u0430\u0439\u0448\u0438\u0435 5 \u043B\u0435\u0442 \xAB\u0418\u043D\u0442\u0435\u0440\u043D\u0435\u0442 \u0432\u0435\u0449\u0435\u0439\xBB \u043F\u043E\u0441\u043F\u043E\u0441\u043E\u0431\u0441\u0442\u0432\u0443\u0435\u0442 \u043F\u043E\u0432\u044B\u0448\u0435\u043D\u0438\u044E \u043E\u0431\u0440\u0430\u0437\u0430 \u0438 \u043A\u0430\u0447\u0435\u0441\u0442\u0432\u0430 \u0436\u0438\u0437\u043D\u0438 \u0447\u0435\u043B\u043E\u0432\u0435\u0447\u0435\u0441\u0442\u0432\u0430." #@param {type:"string"}
stexts = texts
print(texts)

Как «Интернет вещей» изменит нашу жизнь через 5 лет Опубликовано: 23/11/2019Автор: Андрей ЛапушкинКатегория: ОбзорыОставить комментарий Internet of thing – IOT – Интернет вещей обложка Содержание	 Что такое интернет вещей Как технология упрощает жизнь Как работает технология IoT (internet of things) Главная особенность «Интернета вещей»  Применение IoT-технологии в разных секторах Общественный сектор или устройство города Аграрный сектор Сектор логистики Строительный сектор Что такое «умный дом» Развитие «умной медицины» в течение пяти лет «Умный ритейл» или как будут выглядеть магазины будущего Носимые устройства Умные машины «Internet of things» — ожидания и реальность Что такое интернет вещей Интернет вещей это — сеть физических предметов («вещей»), которые оснащены встроенной технологией взаимодействия друг с другом и внешней средой. Организация таких сетей рассматривается как явление, способное перестроить как экономические, так и общественные процессы посредством исключения из ча

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [3]:
def printTF_IDF(texts):
  records_count = 30
  tfIdfTransformer = TfidfVectorizer(ngram_range=(1, 4), use_idf=True, max_features=records_count)
  countVectorizer = CountVectorizer(ngram_range=(1, 4), max_features=records_count)
  wordCount = countVectorizer.fit_transform([texts])
  TfIdf = tfIdfTransformer.fit_transform([texts])
  names = countVectorizer.get_feature_names()
  df=[]
  df = pd.DataFrame(list(names), columns=['names'])
  df = df.assign(Word_Count=wordCount.T.todense())
  df = df.assign(TF_IDF=TfIdf.T.todense())
  df = df.sort_values('TF_IDF', ascending=False)
  print(df)

In [4]:
printTF_IDF(texts)

             names  Word_Count    TF_IDF
6            вещей          29  0.397260
11        интернет          26  0.356164
2              iot          25  0.342466
12  интернет вещей          25  0.342466
21      технология          17  0.232877
16              на          16  0.219178
26        человека          15  0.205479
29             это          13  0.178082
10             или          13  0.178082
25      устройства          12  0.164384
13             как          12  0.164384
28             что          11  0.150685
22           умные          11  0.150685
23           умный          10  0.136986
20      технологии           9  0.123288
27           через           8  0.109589
14         которые           8  0.109589
3   iot технологии           8  0.109589
18          сектор           8  0.109589
24       умный дом           7  0.095890
15             лет           7  0.095890
19           собой           7  0.095890
17              не           7  0.095890
9               

удалим знаки переноса строк и сделаем текст в нижнем регистре

In [5]:
  str2 = ''
  for item in texts.split():
    str2 = str2 + ' ' + item
  texts = str(texts)
  texts = texts.lower()
  # print(texts)
  texts = texts.replace('\n', ' ')

удалим цифры

In [6]:
  str2 = ''
  for c in texts:
      if c not in ('0', "1", '2', '3', '4', '5', '6', '7', '8', '9', '«', '»', '–', "\""):
          str2 = str2 + c
  texts = str2
  str2 = ''

удалим знаки пунктуации

In [7]:
import string
pattern = string.punctuation
for c in texts:
    if c not in pattern:
        str2 = str2 + c
    else:
        str2 = str2 + " "
texts = str2
str2 = ''

удалим стоп слова

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
  from nltk.corpus import stopwords
  russian_stopwords = stopwords.words("russian")
  for word in texts.split():
      if word not in (russian_stopwords):
          str2 = str2 + " " + word
  texts = str2

  str2 = ''
  for word in texts.split():
      if len(word) > 1:
          str2 = str2 + " " + word
  texts = str2
  print(texts)

 интернет вещей изменит нашу жизнь лет опубликовано автор андрей лапушкинкатегория обзорыоставить комментарий internet of thing iot интернет вещей обложка содержание такое интернет вещей технология упрощает жизнь работает технология iot internet of things главная особенность интернета вещей применение iot технологии разных секторах общественный сектор устройство города аграрный сектор сектор логистики строительный сектор такое умный дом развитие умной медицины течение пяти лет умный ритейл будут выглядеть магазины будущего носимые устройства умные машины internet of things ожидания реальность такое интернет вещей интернет вещей это сеть физических предметов вещей которые оснащены встроенной технологией взаимодействия друг другом внешней средой организация таких сетей рассматривается явление способное перестроить экономические общественные процессы посредством исключения части действий операций необходимость участия человека технология упрощает жизнь данная технология автоматизирует раз

рассчитаем TF-IDF

In [10]:
printTF_IDF(texts)

               names  Word_Count    TF_IDF
8              вещей          29  0.423189
12          интернет          26  0.379410
2                iot          25  0.364818
13    интернет вещей          25  0.364818
20        технология          17  0.248076
28          человека          15  0.218891
29               это          13  0.189705
25        устройства          12  0.175113
21             умные          11  0.160520
22             умный          10  0.145927
19        технологии           9  0.131334
17            сектор           8  0.116742
3     iot технологии           8  0.116742
14           которые           8  0.116742
23         умный дом           7  0.102149
18             собой           7  0.102149
15               лет           7  0.102149
11               дом           7  0.102149
7              будут           7  0.102149
1        internet of           6  0.087556
10           датчики           6  0.087556
6          благодаря           6  0.087556
24         

#Суммаризации текстов 

Задача суммаризации текстов (автореферирование) - одна из ключевых, широко обсуждаемых задач NLP. Она состоит в сжатии больших объемов текста до связного краткого содержания, отражающего только основные идеи.


#Алгоритм TextRank
TextRank - это алгоритм, основанный на PageRank, который часто используется для извлечения ключевых слов и суммирования текста. 
PageRank (PR) - это алгоритм, используемый для расчета веса веб-страниц. 


In [11]:
from gensim.summarization import summarize, keywords
print("Исходный текст:")
print(stexts)
print("Результат работы TextRank:")
print(summarize(str(stexts)))

Исходный текст:
Как «Интернет вещей» изменит нашу жизнь через 5 лет Опубликовано: 23/11/2019Автор: Андрей ЛапушкинКатегория: ОбзорыОставить комментарий Internet of thing – IOT – Интернет вещей обложка Содержание	 Что такое интернет вещей Как технология упрощает жизнь Как работает технология IoT (internet of things) Главная особенность «Интернета вещей»  Применение IoT-технологии в разных секторах Общественный сектор или устройство города Аграрный сектор Сектор логистики Строительный сектор Что такое «умный дом» Развитие «умной медицины» в течение пяти лет «Умный ритейл» или как будут выглядеть магазины будущего Носимые устройства Умные машины «Internet of things» — ожидания и реальность Что такое интернет вещей Интернет вещей это — сеть физических предметов («вещей»), которые оснащены встроенной технологией взаимодействия друг с другом и внешней средой. Организация таких сетей рассматривается как явление, способное перестроить как экономические, так и общественные процессы посредством 

#Алгоритм Rake
RAKE: Rapid Automatic Keyword Extraction Algorithm
Алгоритм RAKE извлекает ключевые слова с помощью основанного на разделителе подхода, чтобы идентифицировать ключевые слова кандидата и баллы их использующий совместные встречаемости слова, которые появляются в ключевых словах кандидата. Ключевые слова могут содержать несколько лексем. Кроме того, алгоритм RAKE также объединяет ключевые слова, когда они кажутся многократно, разделенными тем же разделителем слияния.


In [12]:
!pip install rake_nltk
from rake_nltk import Metric, Rake
r = Rake(language="russian")
r.extract_keywords_from_text(stexts)
mas = r.get_ranked_phrases()
set2 = set()
for item in mas:
    if not "nan" in str(item).replace(" nan ", " "):
        set2.add(str(item).replace(" nan ", " "))
mas = list(set2)
print("Исходный текст:")
print(stexts)
print("Результат работы TextRammk:")
print(str(mas))

Исходный текст:
Как «Интернет вещей» изменит нашу жизнь через 5 лет Опубликовано: 23/11/2019Автор: Андрей ЛапушкинКатегория: ОбзорыОставить комментарий Internet of thing – IOT – Интернет вещей обложка Содержание	 Что такое интернет вещей Как технология упрощает жизнь Как работает технология IoT (internet of things) Главная особенность «Интернета вещей»  Применение IoT-технологии в разных секторах Общественный сектор или устройство города Аграрный сектор Сектор логистики Строительный сектор Что такое «умный дом» Развитие «умной медицины» в течение пяти лет «Умный ритейл» или как будут выглядеть магазины будущего Носимые устройства Умные машины «Internet of things» — ожидания и реальность Что такое интернет вещей Интернет вещей это — сеть физических предметов («вещей»), которые оснащены встроенной технологией взаимодействия друг с другом и внешней средой. Организация таких сетей рассматривается как явление, способное перестроить как экономические, так и общественные процессы посредством 

#Тематическое моделирование
Тематическое моделирование — это метод извлечения тем из текста. Latent Dirichlet Allocation (LDA) — популярный алгоритм моделирования тем реализованные в том числе в пакете Gensim. Основная задача алгоритмов ТМ, заключается в том что бы полученные темы были хорошего качество, понятными, самозначимыми и разделенными. Достижение этих целей во многом зависит от качества предварительной обработки текста и стратегии поиска оптимального количества тем. 

##Импорт пакетов


In [13]:
import nltk; nltk.download('stopwords')
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Подход LDA к тематическому моделированию заключается в том, что каждый документ рассматривается как набор тем в определенной пропорции. И каждая тема как набор ключевых слов, опять же, в определенной пропорции.

После того, как вы укажете алгоритму количество тем, все, что он сделает, — это отобразит распределение тем в документах и распределение ключевых слов по темам.

Тема — это не что иное, как набор доминирующих ключевых слов. Просто взглянув на ключевые слова, вы сможете определить, о чем эта тема.

Ниже приведены ключевые факторы для получения хороших разделительных тем:

Качество обработки текста.
Разнообразие тем, о которых говорится в тексте.
Выбор алгоритма моделирование тем.
Количество тем, указанных в алгоритме.
Алгоритмы настройки параметров.

# Подготовим стоп-слова


In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

##Лемматизация
Лемматизация — это не что иное, как преобразование слова в его корневое слово. Например: лемма слова «machines» — это «machine». Аналогично, «walking» -> «walk», «mice» -> «mouse» и так далее.

In [15]:
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
ltexts = ""
print(texts)
for word in texts.split():
    if len(str(word)) > 2:
        ltexts+=" " + morph.parse(word)[0].normal_form
print(ltexts)

 интернет вещей изменит нашу жизнь лет опубликовано автор андрей лапушкинкатегория обзорыоставить комментарий internet of thing iot интернет вещей обложка содержание такое интернет вещей технология упрощает жизнь работает технология iot internet of things главная особенность интернета вещей применение iot технологии разных секторах общественный сектор устройство города аграрный сектор сектор логистики строительный сектор такое умный дом развитие умной медицины течение пяти лет умный ритейл будут выглядеть магазины будущего носимые устройства умные машины internet of things ожидания реальность такое интернет вещей интернет вещей это сеть физических предметов вещей которые оснащены встроенной технологией взаимодействия друг другом внешней средой организация таких сетей рассматривается явление способное перестроить экономические общественные процессы посредством исключения части действий операций необходимость участия человека технология упрощает жизнь данная технология автоматизирует раз

##Создадим словарь и корпус.
Двумя основными входными данными для тематической модели LDA являются словарь (id2word) и корпус. Давайте создадим их.

In [16]:
from nltk.util import ngrams
def make_bigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    bigrams = list(ngrams(token, 2))
    # print(bigrams)
    return bigrams
def make_trigrams(ttexts):
    texts = ' '.join(ttexts)
    token = nltk.word_tokenize(ttexts)
    trigrams = list(ngrams(token, 2))
    return trigrams

btexts = make_bigrams(texts) + make_trigrams(texts)
id2word = corpora.Dictionary(btexts)
corpus = [id2word.doc2bow(text) for text in btexts]
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('вещей', 1), ('интернет', 1)]]

Или вы можете увидеть удобочитаемую форму самого корпуса.

In [17]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('вещей', 1), ('интернет', 1)]]

##Построим тематическую модель
У нас есть все необходимое для обучения модели LDA. В дополнение к корпусу и словарю необходимо также указать количество тем.

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

##Просмотр тем в модели LDA
Вышеупомянутая модель LDA построена на 20 различных темах, где каждая тема представляет собой комбинацию ключевых слов, и каждое ключевое слово вносит определенный вес в тему.

Вы можете увидеть ключевые слова для каждой темы и вес (важность) каждого ключевого слова, используя lda_model.print_topics().

In [19]:
lda_model.print_topics()

[(0,
  '0.073*"человека" + 0.036*"умные" + 0.033*"особенную" + 0.033*"каждое" + 0.033*"современного" + 0.033*"инфраструктуру" + 0.033*"подчинено" + 0.033*"действие" + 0.026*"будут" + 0.017*"искусственному"'),
 (1,
  '0.110*"вещей" + 0.106*"интернет" + 0.051*"лет" + 0.035*"интеллекту" + 0.018*"поспособствует" + 0.018*"искусственному" + 0.017*"датчики" + 0.014*"время" + 0.010*"наладит" + 0.009*"представление"'),
 (2,
  '0.048*"ближайшие" + 0.025*"машины" + 0.019*"момент" + 0.019*"данный" + 0.017*"устройств" + 0.013*"девайсов" + 0.013*"времени" + 0.012*"интернету" + 0.012*"искусственного" + 0.010*"компании"'),
 (3,
  '0.106*"iot" + 0.047*"технология" + 0.043*"устройства" + 0.042*"технологии" + 0.022*"превратится" + 0.019*"которые" + 0.016*"году" + 0.015*"развитием" + 0.015*"образом" + 0.014*"носимые"'),
 (4,
  '0.038*"образа" + 0.038*"повышению" + 0.038*"жизни" + 0.038*"качества" + 0.035*"это" + 0.021*"умный" + 0.019*"поспособствует" + 0.019*"человечества" + 0.014*"дом" + 0.013*"физически

##Визуализация темы и ключевых слов
Теперь, когда модель LDA создана, следующим шагом является изучение созданных тем и связанных с ними ключевых слов. Нет лучшего инструмента, чем интерактивная диаграмма пакета pyLDAvis.

In [20]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [21]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.107343  0.101273       1        1  21.107713
1      0.085748 -0.292225       2        1  20.265608
3     -0.007623  0.151547       3        1  19.642557
4      0.119162  0.064527       4        1  19.623740
0     -0.304630 -0.025122       5        1  19.360383, topic_info=               Term        Freq       Total Category  logprob  loglift
0             вещей  108.000000  108.000000  Default  30.0000  30.0000
1          интернет  105.000000  105.000000  Default  29.0000  29.0000
15              iot  101.000000  101.000000  Default  28.0000  28.0000
85         человека   69.000000   69.000000  Default  27.0000  27.0000
5               лет   50.000000   50.000000  Default  26.0000  26.0000
..              ...         ...         ...      ...      ...      ...
725           водой    5.704156    6.431038   Topic5  -5.1015   1.5220
729         обороты    5.704156    6.431038   Topic5  -5.1015   1.5220
730            счет    5.704156    6.431038   Topic5  -5.1015   1.5220
768  искусственному   15.576982   33.510041   Topic5  -4.0969   0.8759
761     превратится   15.576982   37.269995   Topic5  -4.0969   0.7695

[191 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
12        5  0.967399       internet
15        3  0.994776            iot
13        5  0.967414             of
22        5  0.930543         things
445       1  0.861671  автоматически
...     ...       ...            ...
85        5  0.983642       человека
110       4  0.943869       человеку
775       4  0.944787   человечества
720       5  0.932975   человечеству
55        4  0.966461            это

[166 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 4, 5, 1])